# Create World Happiness Dataset with Full ETL Pipeline
In this small project I want to showcase how you can create a custom dataset by scraping the web and using other sources like APIs to add more features to your dataset. I also want to show these steps embedded into an Extract Transform Load (ETL) pipeline, as data engineers are using it for creating datasets. 

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import numpy as np
from datetime import datetime

When creating an ETL pipeline which sould be executed in an automated fashion later, then it is always useful to log as much information as possible to make the debugging, in case there occured a problem, easier. <br>
In this project, I create a text file and log the start and end parts of each pipeline steps. In a real ETL pipeline, one could also log information about catched errors or error messages in case there are some.

In [2]:
def log(message):
    # set timestamp format
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    # get current timestamp
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

## Extract
The Extract part is the first step of the pipeline. In this step, the data is extracted from all different sources and cached. In this project, I want to extract data from the following sources: <br>
1. Wikipedia -> Scraping the World Happiness Report table
2. Rapid API -> Getting the population data for each country in the Wikipedia dataset
3. Worlddata Website -> Get data about the median age per country

In [3]:
# create log message
log("Start Extract Part")

### Webscraping Happiness Data of Countries
Let's now scrape the Wikipedia page containing the world happiness report from the following link: https://en.wikipedia.org/wiki/World_Happiness_Report

In [4]:
# create log message
log("Start Webscraping Wikipedia Happiness Data")

In [5]:
# get html data first
html_data = requests.get("https://en.wikipedia.org/wiki/World_Happiness_Report")

# check if status is 200 -> shows that its allowed to scrape the webpage
print(html_data.status_code)

200


In [6]:
# parse html data now using BeautifulSoup
soup = BeautifulSoup(html_data.text, "html.parser")

# get all tables from wikipedia page
tables = soup.find_all('table',{'class':"wikitable"})

# store target table
table = tables[0]

# convert table html code to pandas df
data = pd.read_html(str(table))
df_happiness = pd.DataFrame(data[0]) 

# show head
df_happiness.head()

,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,1,Finland,7.809,1.285,1.500,0.961,0.662,0.160,0.478
1,2,Denmark,7.646,1.327,1.503,0.979,0.665,0.243,0.495
2,3,Switzerland,7.560,1.391,1.472,1.041,0.629,0.269,0.408
3,4,Iceland,7.504,1.327,1.548,1.001,0.662,0.362,0.145
4,5,Norway,7.488,1.424,1.495,1.008,0.670,0.288,0.434


In [19]:
# rename some countries to later match the country names from RapidAPI
df_happiness = df_happiness.apply(lambda x: x.replace("Congo (Kinshasa)", "DR Congo"))
df_happiness = df_happiness.apply(lambda x: x.replace("Congo (Brazzaville)", "Congo"))
df_happiness = df_happiness.apply(lambda x: x.replace("Ivory Coast", "Côte d'Ivoire"))

In [20]:
# print number of countries in the dataset
list_countries = df_happiness["Country or region"].to_list()
print(f"Number of countries in Wikipedia dataset: {len(list_countries)}")

Number of countries in Wikipedia dataset: 153


In [21]:
# create log message
log("Finished Webscraping Wikipedia Happiness Data")

### Fetch Population Data for each Country from RapidAPI
Let's now get the population data for each country of our countries dataset. For this I searched on RapidAPI if there is an API for this purpose. And I found this one: https://rapidapi.com/aldair.sr99/api/world-population/.
This is perfect for this project!

In [23]:
# create log message
log("Start Getting Population Data per Country from RapidAPI")

In [24]:
from tqdm import tqdm

# create URL and headers for API call
url = "https://world-population.p.rapidapi.com/population"

# the headers can be found when logging in to your RapidAPI account and opening the link above
headers = {
    'x-rapidapi-host': "world-population.p.rapidapi.com",
    'x-rapidapi-key': "***************************"
}

# add population column first by setting all values to NaN
df_happiness["Population"] = np.nan

# loop over countries and get population
for country in tqdm(df_happiness["Country or region"].to_list()):
    # create querystring for API call
    querystring = {"country_name" : country}
    
    # create request and fetch response
    response = requests.request("GET", url, headers=headers, params=querystring)
    
    # add population to dataframe in case response is okay
    response_dict = json.loads(response.text)
    if response_dict["ok"] == True:
        population = response_dict["body"]["population"]
        df_happiness.loc[df_happiness["Country or region"] == country, "Population"] = population

100%|████████████████████████████████████████████████████████████████████████████████| 153/153 [01:18<00:00,  1.95it/s]


In [25]:
df_happiness.head()

,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Population
0,1,Finland,7.809,1.285,1.500,0.961,0.662,0.160,0.478,5540720.0
1,2,Denmark,7.646,1.327,1.503,0.979,0.665,0.243,0.495,5792202.0
2,3,Switzerland,7.560,1.391,1.472,1.041,0.629,0.269,0.408,8654622.0
3,4,Iceland,7.504,1.327,1.548,1.001,0.662,0.362,0.145,341243.0
4,5,Norway,7.488,1.424,1.495,1.008,0.670,0.288,0.434,5421241.0


In [26]:
df_happiness.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Overall rank                  153 non-null    int64  
 1   Country or region             153 non-null    object 
 2   Score                         153 non-null    float64
 3   GDP per capita                153 non-null    float64
 4   Social support                153 non-null    float64
 5   Healthy life expectancy       153 non-null    float64
 6   Freedom to make life choices  153 non-null    float64
 7   Generosity                    153 non-null    float64
 8   Perceptions of corruption     153 non-null    float64
 9   Population                    148 non-null    float64
dtypes: float64(8), int64(1), object(1)
memory usage: 12.1+ KB


In [27]:
# create log message
log("Finished Getting Population Data per Country from RapidAPI")

Okay nice! There are 5 missing population numbers. But let's ignore them now. This could no be seen as part of the ML engineer to impute the missing data here. 

### Scrape Webpage for getting Average Age per Country
Let's now scrape another webpage and get the average age by country. For this purpose, the webpage which can be found on the following link, is scraped: https://www.worlddata.info/average-age.php

In [28]:
# create log message
log("Start Getting Average Age Data per Country from Worlddata")

In [29]:
# get html data first
html_data = requests.get("https://www.worlddata.info/average-age.php")

# check if status is 200 -> shows that its allowed to scrape the webpage
print(html_data.status_code)

200


In [30]:
# parse html data now using BeautifulSoup
soup = BeautifulSoup(html_data.text, "html.parser")

# get all tables from wikipedia page
tables = soup.find_all('table',{'class':"std100 hover"})

# store target table
table = tables[0]

# convert table html code to pandas df
data = pd.read_html(str(table))
df_average_age = pd.DataFrame(data[0]) 

# show head
df_average_age.head()

,Country,Median agein years,Population under20 years old,Life expectancyin years
0,Japan,48.6,17.0 %,84.5
1,Germany,47.8,18.9 %,81.0
2,Italy,46.5,17.7 %,83.3
3,Hong Kong,45.6,16.2 %,85.4
4,Greece,45.3,18.7 %,82.0


In [31]:
# print number of countries in the dataset
list_countries = df_average_age["Country"].to_list()
print(f"Number of countries in the average age dataset: {len(list_countries)}")

Number of countries in the average age dataset: 128


Okay. There are 116 different countries in this dataset, but our happiness dataset contains 153. So there is some data missing for some countries. <br>
Let's now add these columns to the dataset.

In [32]:
# let's use pandas join functionality for joining these tables together
df_final = df_happiness.set_index("Country or region").join(df_average_age.set_index("Country")).reset_index()
df_final.head()

,Country or region,Overall rank,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Population,Median agein years,Population under20 years old,Life expectancyin years
0,Finland,1,7.809,1.285,1.500,0.961,0.662,0.160,0.478,5540720.0,42.8,21.2 %,81.9
1,Denmark,2,7.646,1.327,1.503,0.979,0.665,0.243,0.495,5792202.0,42.0,22.1 %,81.3
2,Switzerland,3,7.560,1.391,1.472,1.041,0.629,0.269,0.408,8654622.0,42.7,19.9 %,83.8
3,Iceland,4,7.504,1.327,1.548,1.001,0.662,0.362,0.145,341243.0,37.1,25.7 %,82.6
4,Norway,5,7.488,1.424,1.495,1.008,0.670,0.288,0.434,5421241.0,39.5,23.2 %,82.9


In [33]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Country or region             153 non-null    object 
 1   Overall rank                  153 non-null    int64  
 2   Score                         153 non-null    float64
 3   GDP per capita                153 non-null    float64
 4   Social support                153 non-null    float64
 5   Healthy life expectancy       153 non-null    float64
 6   Freedom to make life choices  153 non-null    float64
 7   Generosity                    153 non-null    float64
 8   Perceptions of corruption     153 non-null    float64
 9   Population                    148 non-null    float64
 10  Median agein years            103 non-null    float64
 11  Population under20 years old  103 non-null    object 
 12  Life expectancyin years       103 non-null    float64
dtypes: fl

In [34]:
# create log message
log("Finished Getting Average Age Data per Country from Worlddata")

log("Extract Part of Pipeline Finished")

Okay. So there are now more missing values. As a future step, one could try to also get these missing values. But in this project, I only quickly wanted to showcase how such an ETL piplien could look like. So here I am not trying to find the missing values for the missing countries. Feel free to extend this project and dataset if you want to!

## Transform
Let's now start with the transform part. Here the data is transformed such that it is in the expected format. The datatypes are already in the expected format. Let's first add the total GDP per country. This is easy, as long as the population is given, because we already have the GDP per capita. So the total GDP can be computed this way: <br> <br>
$GDP = GDP per capita \times Population$ <br> <br>
Let's transform the Population under 20 years old column to be of type float, as the second transform step.

In [35]:
# create log message
log("Start Transform Part of Pipeline")

In [36]:
df_final["GDP"] = df_final["GDP per capita"] * df_final["Population"]
df_final.head()

,Country or region,Overall rank,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Population,Median agein years,Population under20 years old,Life expectancyin years,GDP
0,Finland,1,7.809,1.285,1.500,0.961,0.662,0.160,0.478,5540720.0,42.8,21.2 %,81.9,7.119825e+06
1,Denmark,2,7.646,1.327,1.503,0.979,0.665,0.243,0.495,5792202.0,42.0,22.1 %,81.3,7.686252e+06
2,Switzerland,3,7.560,1.391,1.472,1.041,0.629,0.269,0.408,8654622.0,42.7,19.9 %,83.8,1.203858e+07
3,Iceland,4,7.504,1.327,1.548,1.001,0.662,0.362,0.145,341243.0,37.1,25.7 %,82.6,4.528295e+05
4,Norway,5,7.488,1.424,1.495,1.008,0.670,0.288,0.434,5421241.0,39.5,23.2 %,82.9,7.719847e+06


In [37]:
# let's now remove the % sign of the Population under 20 years old column and convert it to type float
def transform_col(col_val):
    try: 
        return float(col_val.replace(" %", ""))
    except: # value is NaN
        return col_val

df_final["Population under 20 years old in %"] = df_final["Population under20 years old"].apply(transform_col)
df_final = df_final.drop(columns=["Population under20 years old"])
df_final.head()

,Country or region,Overall rank,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Population,Median agein years,Life expectancyin years,GDP,Population under 20 years old in %
0,Finland,1,7.809,1.285,1.500,0.961,0.662,0.160,0.478,5540720.0,42.8,81.9,7.119825e+06,21.2
1,Denmark,2,7.646,1.327,1.503,0.979,0.665,0.243,0.495,5792202.0,42.0,81.3,7.686252e+06,22.1
2,Switzerland,3,7.560,1.391,1.472,1.041,0.629,0.269,0.408,8654622.0,42.7,83.8,1.203858e+07,19.9
3,Iceland,4,7.504,1.327,1.548,1.001,0.662,0.362,0.145,341243.0,37.1,82.6,4.528295e+05,25.7
4,Norway,5,7.488,1.424,1.495,1.008,0.670,0.288,0.434,5421241.0,39.5,82.9,7.719847e+06,23.2


In [38]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 14 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Country or region                   153 non-null    object 
 1   Overall rank                        153 non-null    int64  
 2   Score                               153 non-null    float64
 3   GDP per capita                      153 non-null    float64
 4   Social support                      153 non-null    float64
 5   Healthy life expectancy             153 non-null    float64
 6   Freedom to make life choices        153 non-null    float64
 7   Generosity                          153 non-null    float64
 8   Perceptions of corruption           153 non-null    float64
 9   Population                          148 non-null    float64
 10  Median agein years                  103 non-null    float64
 11  Life expectancyin years             103 non-n

Perfect! Now we also finished the transform part!

In [39]:
# create log message
log("Finished Transform Part of Pipeline")

## Load
The last step of the pipeline is to load the dataset into the expected final format (i.e. a SQL database, ...). For the sake of this project, I am loading the final dataset into a csv file. 

In [40]:
# create log message
log("Start Load Part of Pipeline")

In [41]:
def load(dataset):
    dataset.to_csv("final_dataset.csv", index=False)

load(df_final)

In [42]:
# create log message
log("Finished Load Part of Pipeline")